In [1]:
import os
import cv2 as cv
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, ReLU, Add, Concatenate
from tensorflow.keras.models import Model
from math import log10, sqrt 
import tensorflow as tf

def load_images_from_folder(foldername, target_size=(64, 64)):
    images = []
    for filename in os.listdir(foldername):
        img = cv.imread(os.path.join(foldername, filename), cv.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv.resize(img, target_size)  
            images.append(img)
    return images


# PSNR function from previous submission, modified for Tensor use 
def PSNR(original, compressed):
    original = tf.cast(original, tf.float32)
    compressed = tf.cast(compressed, tf.float32)      
    mse = tf.reduce_mean(tf.square(original - compressed))
    max_pixel = 255.0
    psnr = 20 * tf.math.log(max_pixel / tf.sqrt(mse)) / tf.math.log(10.0)

    return psnr 

input_img = Input(shape=(64, 64, 1))  

def res_block(input):
    x = Conv2D(32, (3,3), activation='relu', padding='same')(input)
    x = BatchNormalization()(x)
    x = Conv2D(64, (2,2), padding='same')(x)
    out = Add()([input,x])
    out = ReLU()(out)
    out = BatchNormalization()(out)
    return out

def inception_block(input, ):
    a = Conv2D(32, (1, 1), padding='same', activation='relu')(input)
    b = Conv2D(64, (3, 3), padding='same', activation='relu')(input)
    c = Conv2D(64, (5, 5), padding='same', activation='relu')(input)
    d = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input)
    d = Conv2D(32, (1, 1), padding='same', activation='relu')(d)

    output = Concatenate(axis=-1)([a,b,c,d])
    return output

def encode (input):
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    return encoded
def decode (input):
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    return decoded

def res_forward(input):
    encoded = encode(input)
    res = res_block(encoded)
    decoded = decode(res)
    return decoded

def inception_forward(input):
    encoded = encode(input)
    inception = inception_block(encoded)
    decoded = decode(inception)
    return decoded

# Build autoencoder model
autoencoder_res = Model(input_img, res_forward(input_img))
autoencoder_res.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder_inception = Model(input_img, inception_forward(input_img))
autoencoder_inception.compile(optimizer='adam', loss='binary_crossentropy')


# Load images
images=load_images_from_folder("pneumonia")
images = np.array(images)
images = images.astype('float32') / 255.
images = np.expand_dims(images, axis=-1)

# Generate noisy images
lam25noise = np.random.poisson(25, images.shape)
lam25 = images + lam25noise

lam50noise = np.random.poisson(50, images.shape)
lam50 = images + lam50noise

lam75noise = np.random.poisson(75, images.shape)
lam75 = images + lam75noise


# Split the dataset into training and testing sets (80% train, 20% test)
x_train, x_test = train_test_split(images, test_size=0.2, random_state=42)
train25, test25 = train_test_split(lam25, test_size=0.2, random_state=42)
train50, test50 = train_test_split(lam50, test_size=0.2, random_state=42)
train75, test75 = train_test_split(lam75, test_size=0.2, random_state=42)

In [12]:
denoised_res_lam25 = autoencoder_res.predict(test25)
psnr_res_lam25 = PSNR(test25, denoised_res_lam25).numpy()
print("PSNR for Residual Skip Connections model on lam25:", psnr_res_lam25)

denoised_inception_lam25 = autoencoder_inception.predict(test25)
psnr_inception_lam25 = PSNR(test25, denoised_inception_lam25).numpy()
print("PSNR for Inception Network model on lam25:", psnr_inception_lam25)

15/15 [==============================] - 0s 23ms/step
PSNR for Residual Skip Connections model on lam25: 20.177284
15/15 [==============================] - 1s 37ms/step
PSNR for Inception Network model on lam25: 20.04568


#### 1) Which approach performed better for this particular task of denoising?

Based on the PSNR (Peak Signal-to-Noise Ratio) values obtained for the lam25 noisy images:
- PSNR for Residual Skip Connections model on lam25: 20.177284
- PSNR for Inception Network model on lam25: 20.04568

It can be observed that the Residual Skip Connections model achieved a higher PSNR value as compared to the Inception Network model. Since PSNR measures the quality of the denoised image wherein a higher value indicates better denoising, it can be concluded that Residual Blocks performed better for denoising the lambda 25 noisy images in this particular task.

#### 2. What is the intuition behind the improved model which made it ideal for this particular task?

The improved model with Residual skip connections involve bypassing one or more intermediate layers by directly adding the input of a layer to its output. This allows the network to learn residual mappings, capturing the difference between the input and the output of a layer. By facilitating the flow of gradients during training and preserving useful information from earlier layers, residual skip connections enable the training of deeper networks and help mitigate the vanishing gradient problem. This mechanism contributes to improved learning efficiency and model performance in various tasks, including image denoising.

On the other hand, the improved model with Inception blocks is ideal for denoising tasks due to its ability to capture features at multiple scales simultaneously. By incorporating convolutional filters of varying sizes and parallel pathways for feature extraction, the model can effectively preserve important details while removing noise. This adaptive combination of information and hierarchical representation learning enables the model to capture complex patterns present in noisy images, contributing to enhanced denoising performance.